### imports and preparation

- place this notebook in ../typeform 
- store your own token for tf API in a file called "typeform_token.py" and save it locally (do not commit) in /typeform
- The file contains only one line: TOKEN="your_token"

In [ ]:
import requests
import pandas as pd
import time
import textwrap

#import my own function
%run one_entry_to_dictionary.py
%run typeform_token.py

In [ ]:
import sys
sys.path.append('../')

# 1. Pull Data from Typeform

### Connect to TF, see metainfo for form

In [ ]:
#IMPORT TOKEN FILE 
auth_token= TOKEN
head = {'Authorization': 'Bearer ' + auth_token}
url = 'https://api.typeform.com/forms/OHziDn'

metainfo_form = requests.get(url, headers=head)
#response 200 means its good !
print(metainfo_form)
print(metainfo_form.json())

### Pull all responses exclusing mail from Typeform

In [ ]:
# get all answers but not emails
SIZE = "500"
url = "https://api.typeform.com/forms/OHziDn/responses?fields=Xa2T0bcZ5VnY,QUtA6k0gcrZf,ZNZyNVwMtwID,azlKRkvdcc7O,Vrzj2uv0i29C,Xh2W1d6FhXyK,t9EaTsONnJfR,OaujoVSY6NaS,ASolp4v3adZK,erHBE6gKxycf,ASQmq4h3vxRt,KEGztC8iYfyy,kWb7LUjdJyhb,g9goY6hIxmC9,jG2wzvAmOWxq,HNqA7BK7ZDFl&page_size="+SIZE

response = requests.get(url, headers=head)
#response 200 means its good !
print(response)

responses = response.json()

In [ ]:
print("number of responses",responses['total_items'])
#look at the first response
print("the last response:",responses["items"][0])

# 2. Transform JSON to pandas.df and map dummy vars

### step by step

In [ ]:
# LOAD ONE PERSON FROM THE JSON RESPONSE
one_person = responses["items"][0]

In [ ]:
#transform ONE person's information to a dictionary using function one_entry_to_dictionary
dictionary_test = one_entry_to_dictionary(one_person)
print(dictionary_test)

In [ ]:
# transform dictionary to df 
df_test = pd.DataFrame(dictionary_test,index=[0])
df_test.head()

### apply this to entire JSON

In [ ]:
def transform_all_to_df(responses):
    '''
    transforms the JSON response from the above query into a df with cols=vars, rows=individuals
    '''
    no_df_defined = True
    for item in responses["items"]:
        dictionary_transformation = one_entry_to_dictionary(item)
        df = pd.DataFrame(dictionary_transformation,index=[0])
        #crate dictionary and then append to it 
        if no_df_defined:
            df_all = df
            no_df_defined = False
        else: 
            df_all = df_all.append(df)
        df_all = df_all.reset_index(drop = True)
    return df_all

In [ ]:
df_all = transform_all_to_df(responses)

In [ ]:
df_all.head()

In [ ]:
len(df_all)

In [ ]:
df_all.columns

In [ ]:
# If you want to save the data 
"""
timestr = time.strftime("%Y%m%d%H%M")
file_name = "survey_data_" + timestr
print(file_name)

df_all.to_csv("../data/" + file_name + ".csv")

"""

# Aggregation for Graph 0 - Map 

In [ ]:
%load_ext autoreload
%autoreload 2
from overview_tab.map_graph import merge_local_internat_dataframe

In [ ]:
postal_code = pd.read_csv("../data/postcode_uk.csv", index_col = 0)
#countries = pd.read_csv("https://raw.githubusercontent.com/rjcnrd/domestic_violence_covid-19/master/data/countries.csv")
countries = pd.read_csv("../data/countries.csv", index_col = 0)

In [ ]:
all_reports_df, safety_df, safety_change_df, mental_health_df, working_situation_df, not_in_map, uk_data, uk_data_county = merge_local_internat_dataframe(survey_df = df_all, postal_code_df = postal_code, countries_df = countries,  map_threshold = 0, postal_code_col = "location")

In [ ]:
all_reports_df.to_csv("../data/data_map_all_reports.csv")
safety_df.to_csv("../data/data_map_safety_scale.csv")
safety_change_df.to_csv("../data/data_map_safety_change.csv")
mental_health_df.to_csv("../data/data_map_mental_health.csv")
working_situation_df.to_csv("../data/data_map_working_situation.csv")

## Check for the rows that are not integrated

In [ ]:
#Here are the values that are not integrated, by order
not_in_map.location.value_counts()

In [ ]:
#If you want to append new location, you should update the countries csv, one by one 
#country_name : name of the country as you see it in the survey
#Area : name of the country as it appears already in the survey. THIS WILL BE USED FOR A MERGE
#Area_name : name of the country as it should appear in the hover (the same name as area name)
"""
countries_new = countries.append({"latitude": 22.396428, 
                                  "longitude": 114.10949699999999, 
                                  "country_name" : "HKSAR",
                                  "area": "HONG KONG", 
                                  "area_name": "Hong Kong"}, ignore_index=True)
countries_new.to_csv("/Users/ameliemeurer/Documents/Amélie/HEC M2/07. Research Paper - UN Women/02. Code/domestic_violence_covid-19/data/countries.csv")
                                  """


# Aggregation for Graph 1. Scatter-Testimonial


In [ ]:
%load_ext autoreload
%autoreload 2
from statistics_tab.scatter_bar_plot_mental_health import location_scatterplot, merge_testimonials, testimonial_treatment

In [ ]:
#Create the location for all the dots
location_data = location_scatterplot(df_all, do_percentage = False, include_other=False, include_man = True, num_by_col=5)
location_data.head()

In [ ]:
#Create the treatment for testimonials 
MAXIMUM_TOTAL_LENGTH = 150 #Maximum length of testimonial
MAXIMUM_LINE_LENGTH = 30 #Maximum length of the line on hover
MINIMUM_TOTAL_LENGTH = 10 #Minimum length of testimonial
testimonial_treated = testimonial_treatment(df_all[["gender", "mental_scale","testimonial"]], 
                                            maximum_total_length = MAXIMUM_TOTAL_LENGTH, 
                                            maximum_line_length = MAXIMUM_LINE_LENGTH, 
                                            minimum_total_length = MINIMUM_TOTAL_LENGTH)
testimonial_treated.head()

In [ ]:
#Merges the 2 dataframes
data_scatterplot = merge_testimonials(location_data, testimonial_treated, include_other = False)
data_scatterplot.tail()

In [ ]:
data_scatterplot.to_csv("data_scatterplot.csv")